In [ ]:
import pandas as pd
import pickle
import re
import random

ground_truth=pd.read_excel("../data/data_corrected_english.xlsx",index_col="Report")
ground_truth=ground_truth[ground_truth["Eliminar"]!="Yes"]

with open("../data/report_data_q_a_ingles_v2.pkl", 'rb') as file:  # 'rb' mode is for reading binary files
    report_data = pickle.load(file)

def fix_brackets_spaces(texto):
    ''' 
        Introduce espacios por delante y por detrás de los paréntesis.
        Esta medida mejora el tokenizado de Spacy
    '''
    
    texto = re.sub(r'([(\[¿!])', r' \1', texto)
    texto = re.sub(r'([)\]?¡])', r'\1 ', texto)
                
    return texto

def preprocess_text(text):
    text= fix_brackets_spaces(text)
    return text

def visualize_errors(valid_dataset,valid_targets,validation_predictions,keys):
    # Crear un DataFrame con los textos originales, las etiquetas reales y las predicciones
    results_df = pd.DataFrame({
        'key':list(keys),
        'Text': list(valid_dataset),  # Usamos los textos originales
        'True Label': list(valid_targets),
        'Predicted Label': list(validation_predictions)
    })
    
    # Filtrar los ejemplos en los que el modelo falló
    errors_df = results_df[results_df['True Label'] != results_df['Predicted Label']]
    
    for ind,row in errors_df.iterrows():
        print(row["key"])
        print("EJEMPLO")
        print(row["Text"])
        print("PREDICTED")
        print(row["Predicted Label"])
        print("TRUE")
        print(row["True Label"])


def test_train_split(i, X, Y):
    random.seed(1)
    # Agrupar ejemplos originales y sus copias
    def obtener_grupo(nombre):
        # Extraer el nombre base eliminando '_copia' y cualquier número posterior
        if '_copy' in nombre:
            return nombre.split('_copy')[0]

        return nombre

    # Crear un DataFrame temporal para manejar los índices
    agrupaciones = pd.DataFrame(index=X.index)
    agrupaciones['grupo'] = agrupaciones.index.map(obtener_grupo)
    # Obtener una lista de grupos únicos
    grupos_unicos = agrupaciones['grupo'].unique()

    # Dividir grupos en 10 folds
    cv_tamaño = math.ceil(len(grupos_unicos) / 10)
    test_grupos = grupos_unicos[i * cv_tamaño:min(len(grupos_unicos), (i + 1) * cv_tamaño)]
    

    # Filtrar conjuntos de prueba y entrenamiento basados en los grupos
    test_indices = test_grupos
    train_indices = agrupaciones[~agrupaciones['grupo'].isin(test_grupos)].index
    
    # Seleccionar los datos de entrenamiento y prueba
    test = X.loc[test_indices].sort_index()
    test_y = Y.loc[test_indices].sort_index()
    ind=list(test.index)
    
    train = X.loc[train_indices].sort_index()

    # Barajar los datos de entrenamiento para evitar sesgos
    train = train.sample(frac=1, random_state=1)

    train_y = Y.loc[train.index]

    train["label"]=train_y["label"]
    test["label"]=test_y["label"]
    # Devolver los conjuntos
    return train, test, ind
